In [1]:
import requests
import pandas as pd
import numpy as np
import json
import time

ModuleNotFoundError: No module named 'requests'

In [ ]:
apiKey = "IaF6zOUaj8VG4ambmUwOodEFaxBnrR3A"
apiUrl = "https://aeroapi.flightaware.com/aeroapi/"
auth_header = {'x-apikey':apiKey}

# GET flights from DMK airport

In [ ]:
Airport_id = 'DMK'
Airport_start = '2022-12-01T00:00:00Z'
Airport_end = '2022-12-04T00:00:00Z'

# Y-M-D
# 1) 2022-12-01 - 2022-12-04


In [ ]:
# #----- get flight data from DMK airport -----#

response = requests.get(apiUrl + f"airports/{Airport_id}/flights?start=2022-12-26T08%3A00%3A00Z&end=2022-12-26T11%3A00%3A00Z",
 headers=auth_header)

# if response.status_code == 200:
#     print(response.json())
# else:
#     print("Error executing request")

In [ ]:
print(json.dumps(response.json(), indent=4))

In [ ]:
Arrivals_FlightID = []
Departures_FlightID = []

In [ ]:
# tlm_flight = [
#     'TLM100-1671584040-schedule-0562',
#     'TLM100-1671497100-schedule-0570',
#     'TLM100-1671929100-schedule-0825'
# ]
# Departures_FlightID = tlm_flight

In [ ]:
#----- add flight data from DMK airport -----#
for i in response.json()['arrivals']:
    if 'schedule' in i['fa_flight_id']:
        Arrivals_FlightID.append(i['fa_flight_id'])
        
for i in response.json()['departures']:
    if 'schedule' in i['fa_flight_id']:
        Departures_FlightID.append(i['fa_flight_id'])

In [ ]:
for item in Departures_FlightID:
    print(item)

# GET data from flight ID

In [ ]:
#----- get data from flight ID -----#
Dict_flights = {}

i = 1
for id in Departures_FlightID:
    response = requests.get(apiUrl + f"flights/{id}/track",
    headers=auth_header)

    if response.status_code == 200:
        print(i)
        #----- Add data to dict -----#
        Dict_flights[id] = pd.DataFrame.from_dict(response.json()['positions'])
        
        #----- check time -----#
        Dict_flights[id] = Dict_flights[id].drop_duplicates(subset=['timestamp'], keep='first')

        #----- altitude <= 100k ft. & index[1]-index[0]==1 -----#
        Dict_flights[id] = Dict_flights[id][Dict_flights[id]['altitude']<=100]
        Dict_flights[id]['check_index_1'] = Dict_flights[id].index
        Dict_flights[id]['check_index_2'] = Dict_flights[id]['check_index_1'].shift(-1)
        Dict_flights[id]['drop'] = Dict_flights[id]['check_index_2']-Dict_flights[id]['check_index_1'] == 1
            
        #----- loop append only Departures -----#
        res = pd.DataFrame()
        n = 0
        while(Dict_flights[id].iloc[n]['drop']) :
            res = res.append(Dict_flights[id].iloc[n])
            # res = pd.concat([res, Dict_flights[id].iloc[n]])
            n+=1
        res = res.append(Dict_flights[id].iloc[n])
        # res = pd.concat([res, Dict_flights[id].iloc[n]])
        
        #----- interpolate 1 sec -----#
        res['timestamp'] = pd.to_datetime(res.timestamp)
        nidx = np.arange(res['timestamp'][0], res['timestamp'].iloc[-1], 1000000 )
        nidx = pd.to_datetime(nidx)

        res['timestamp'] = res['timestamp'].round('S').dt.tz_localize(None)
        res.set_index('timestamp', inplace=True)
        res = res.reindex(res.index.union(nidx))

        #----- interpolate ['altitude', 'groundspeed', 'heading', 'latitude', 'longitude'] -----#
        res = res[['fa_flight_id', 'altitude', 'groundspeed', 'heading', 'latitude', 'longitude']]
        res['altitude'] = pd.to_numeric(res['altitude'])
        res['groundspeed'] = pd.to_numeric(res['groundspeed'])
        res['heading'] = pd.to_numeric(res['heading'])
        res = res.interpolate(method='time',limit_direction='both',limit=100)
        res['fa_flight_id'] = id
        
        #----- date-time index to timestamp -----#
        res.reset_index(inplace=True)
        res.rename(columns={'index':'timestamp'},inplace=True)
        
        #----- ONLY TEST -----#
        res.to_csv(f'ONLY_TEST/{id}.csv')
            
    else:
        print("Error executing request")
        
    i+=1
    time.sleep(10)

In [ ]:
# Dict_flights['SNY20-1670370011-adhoc-0'].loc[:, ~Dict_flights['SNY20-1670370011-adhoc-0'].index.duplicated()]

In [ ]:
# Dict_flights.keys()

In [ ]:
# print(json.dumps(response.json(), indent=4))

In [ ]:
# df = pd.read_csv(f'flights_data/{id}.csv')
# df.head()

# test ไม่ใช้จ้า

In [ ]:
# Dict_flights = {}

# Dict_flights['AIQ642-1669939380-schedule-0476'] = pd.read_csv(f'flights_data/AIQ642-1669939380-schedule-0476.csv')
# Dict_flights['AIQ642-1669939380-schedule-0476']['fa_flight_id'] = 1

# Dict_flights['NOK340-1669938420-schedule-0407'] = pd.read_csv(f'flights_data/NOK340-1669938420-schedule-0407.csv')
# Dict_flights['NOK340-1669938420-schedule-0407']['fa_flight_id'] = 2

# Dict_flights['AIQ642-1669939380-schedule-0476'].head()

In [ ]:
# Dict_flights['AIQ642-1669939380-schedule-0476']

# Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'] = pd.to_datetime(Dict_flights['AIQ642-1669939380-schedule-0476'].timestamp)

# nidx = np.arange(Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'][0], Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'].iloc[-1], 1000000 )
# nidx = pd.to_datetime(nidx)
# # nidx

# Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'] = Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'].round('S').dt.tz_localize(None)
# Dict_flights['AIQ642-1669939380-schedule-0476'].set_index('timestamp', inplace=True)
# Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'].reindex(Dict_flights['AIQ642-1669939380-schedule-0476'].index.union(nidx))

# Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'][['altitude', 'groundspeed', 'heading', 'latitude', 'longitude']]
# Dict_flights['AIQ642-1669939380-schedule-0476']['altitude'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['altitude'])
# Dict_flights['AIQ642-1669939380-schedule-0476']['groundspeed'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['groundspeed'])
# Dict_flights['AIQ642-1669939380-schedule-0476']['heading'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['heading'])
# Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'].interpolate(method='time',limit_direction='both',limit=50)

# Dict_flights['AIQ642-1669939380-schedule-0476']

In [ ]:
Dict_flights.keys()

In [ ]:
# Dict_flights['SNY20-1670370011-adhoc-0'].reset_index(inplace=True)
# Dict_flights['SNY20-1670370011-adhoc-0']

In [ ]:
Dict_flights['SNY20-1670370011-adhoc-0'] = Dict_flights['SNY20-1670370011-adhoc-0'].drop_duplicates(subset=['timestamp'], keep='first')

In [ ]:
Dict_flights['SNY20-1670370011-adhoc-0'].reset_index(inplace=True)

In [ ]:
Dict_flights['SNY20-1670370011-adhoc-0'].rename(columns={'index':'timestamp'},inplace=True)

In [ ]:
# Dict_flights['NOK600-1670198400-schedule-0700'].between_time('23:00', '07:00')
if Dict_flights['NOK600-1670198400-schedule-0700'].iloc[:1].between_time('23:00', '07:00').empty :
    print(1)
else:
    print(0)
# Dict_flights['NOK600-1670198400-schedule-0700']

In [ ]:
test_dict = {
        'day':{},
        'night':{}
    }

In [ ]:
test_dict['night']['1'] = Dict_flights['SNY20-1670370011-adhoc-0'].copy()

In [ ]:
test_dict['night']['1'][test_dict['night']['1']['altitude'] <= 50]

In [ ]:
test_dict['night']['1']

In [ ]:
for item in Dict_flights:
    print(item)

In [ ]:
Dict_flights[item]

In [ ]:
for time in test_dict:
    print(time)
    for flight in test_dict[time]:
        print(flight)

In [ ]:
time

In [ ]:
Dict_flights['NOK340-1670198400-schedule-0684'][Dict_flights['NOK340-1670198400-schedule-0684']['altitude'] <= 80]

In [ ]:
# SNY20-1670370011-adhoc-0

In [ ]:
tmp = pd.read_csv('ONLY_TEST/AIQ359-1670457540-schedule-0563.csv')
tmp.head()

In [ ]:
tmp = tmp[tmp['altitude'] <= 100]
tmp['check_index_1'] = tmp.index
tmp['check_index_2'] = tmp['check_index_1'].shift(-1)
tmp['drop'] = tmp['check_index_2']-tmp['check_index_1'] == 1

# tmp

In [ ]:
tmp.index

In [ ]:
res = pd.DataFrame()
n = 0
while(tmp.iloc[n]['drop']) :
    res = res.append(tmp.iloc[n])
    n+=1
res = res.append(tmp.iloc[n])

res

In [ ]:
res.rename(columns={'Unnamed: 0':'timestamp'}, inplace=True)
res['timestamp'] = pd.to_datetime(res.timestamp)
res.set_index('timestamp',inplace=True)
res.index.inferred_type == "datetime64"

In [ ]:
res

In [ ]:
res.iloc[:1]

In [ ]:
if res.iloc[:1].between_time('07:00', '23:00').empty :
    print(1)

In [ ]:
res2 = res.reset_index()
res2

In [ ]:
print(res2.iloc[0].timestamp.date())

In [ ]:
res2.timestamp.dt.date

In [ ]:
type(tmp.loc[n])

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
test_df = pd.read_csv(f'flights_data/AIQ642-1669939380-schedule-0476.csv')
test_df

In [ ]:
# order_col = test_df.columns
# order_col

In [ ]:
# order_col = ['groundspeed', 'fa_flight_id', 'update_type']
# order_col = test_df.columns

# test_df[order_col]
# order_col